In [150]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go

# Add the project folder to sys.path
sys.path.append(r'C:\StockAnalyticsProject')
from MyTools.ApiClient import ApiClient
api_client = ApiClient()

dataFolderPath = './'
portfolioFolderPath = os.path.join(dataFolderPath,'etf_portfolio')
portfolioFilePath = os.path.join(portfolioFolderPath, 'portfolio.csv')

risk_free_rate = 0.01

In [151]:

portfolio_df = pd.read_csv(portfolioFilePath,delimiter=';', index_col=0)
#portfolio_data = {symbol: {'quantity': quantity} for symbol, quantity in portfolio_df['quantity'].items()}
print(portfolio_df)
#print(portfolio_data)

         quantity currency  broker
symbol                            
RKLB          100      USD   etoro
PL            650      USD   etoro
VST            20      USD   etoro
ACHR          320      USD   etoro
BKSY          180      USD   etoro
VRT            25      USD   etoro
UEC           370      USD   etoro
GOOG           15      USD   etoro
RDW           150      USD   etoro
AIR.PA         20      EUR  Degiro
BAE.L         100      GBP  Degiro
BSD2.DE       350      EUR  Degiro
GLW            75      USD  Degiro
DBK.DE        100      EUR  Degiro
LDO.MIL        60      EUR  Degiro
TLN             9      USD  Degiro
HO.PA           5      EUR  Degiro
TSSI          100      USD    IBKR
BE            140      USD    IBKR
IDR           150      USD    IBKR
ATRO           70      USD    IBKR
GLJ.DE         50      EUR    IBKR
LHX            10      USD    IBKR
CRC            50      USD    IBKR
VOYG           50      USD    IBKR
UAMY          500      USD    IBKR


In [152]:
portfolio_df = portfolio_df[portfolio_df['currency'] == 'USD']

history_df = pd.DataFrame()
histories=[]

for symbol in portfolio_df.index:
    try:
        code, symbol_historic = api_client.get("getHistory", {"tickerSymbol": symbol})
        # symbol_historic is a tuple, extract the second element (the dict of columns)
        symbol_hist_df = pd.DataFrame({'Date': symbol_historic['Date'], 'Close': symbol_historic['Close']})
        symbol_hist_df['Date'] = pd.to_datetime(symbol_hist_df['Date'], unit='ms').dt.strftime('%Y-%m-%d')
        # Get the last Close value for the current symbol and add it to portfolio_df

        symbol_hist_df['return'] = symbol_hist_df['Close'].pct_change()
        portfolio_df.at[symbol, 'Close'] = symbol_hist_df['Close'].dropna().iloc[-1]
        symbol_hist_df = symbol_hist_df.drop(columns=['Close'])
        # Annualize the mean daily return (r_bar)
        portfolio_df.at[symbol, 'r_bar'] = symbol_hist_df['return'].mean() * 252
        # daily_std is the standard deviation of daily returns, a measure of volatility
        daily_std = symbol_hist_df['return'].std()
        # Yes, std is the square root of variance: std = sqrt(variance)
        # Here, annualized_variance = (daily_std ** 2) * 252, so np.sqrt(annualized_variance) is the annualized std
        annualized_variance = (daily_std ** 2) * 252
        portfolio_df.at[symbol, 'sigma'] = np.sqrt(annualized_variance)
        # Calculate skewness of daily returns for the current symbol
        portfolio_df.at[symbol, 'skewness'] = symbol_hist_df['return'].skew()
        # Calculate kurtosis of daily returns for the current symbol
        portfolio_df.at[symbol, 'kurtosis'] = symbol_hist_df['return'].kurt()
        symbol_hist_df = symbol_hist_df.rename(columns={'return': symbol})
        if history_df.empty:
            history_df = symbol_hist_df
        else:
            history_df = pd.merge(history_df, symbol_hist_df, on='Date', how='outer')

            # Fill missing values with zero except for the 'Date' column
    except Exception as e:
        print(f"Error processing symbol {symbol}: {e}")

history_df.loc[:, history_df.columns != 'Date'] = history_df.loc[:, history_df.columns != 'Date'].fillna(0)
history_df.to_csv(os.path.join(dataFolderPath, 'history_df.csv'), index=False)


Calling http://vaio/getHistory with params: {'tickerSymbol': 'RKLB'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'PL'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'VST'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'ACHR'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'BKSY'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'VRT'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'UEC'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'GOOG'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'RDW'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'GLW'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'TLN'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'TSSI'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'BE'}
Calling http://vaio/getHistory with params: {'tickerSymbol': 'IDR'}
Calling http://vaio/getHistory with params: {

Create a dataframe with the indexes of the last two years:

In [153]:

## create the index dataframe
# Convert 'Date' to datetime for filtering
history_df['Date'] = pd.to_datetime(history_df['Date'])
# Find the latest date in the dataframe
latest_date = history_df['Date'].max()
# Calculate the date 2 years before the latest date
start_date = latest_date - pd.DateOffset(years=2)
# Filter the dataframe for the required date range
recent_df = history_df[history_df['Date'] >= start_date].copy()
# Set 'Date' as index for easier calculations
recent_df.set_index('Date', inplace=True)

for col in recent_df.columns:
    # Create an index that starts at 100 and multiplies by (1 + percentage change) cumulatively
    recent_df[f'i-{col}'] = 100 * (1 + recent_df[col].fillna(0)).cumprod()
    recent_df.drop(columns=[col], inplace=True)

# Reset index to have 'Date' as a column again
index_df = recent_df.reset_index()

index_df 

,Date,i-RKLB,i-PL,i-VST,i-ACHR,i-BKSY,i-VRT,i-UEC,i-GOOG,i-RDW,i-GLW,i-TLN,i-TSSI,i-BE,i-IDR,i-ATRO,i-LHX,i-CRC,i-VOYG,i-UAMY
0,2023-07-25,103.112314,100.299401,99.293293,104.366819,98.974354,100.814579,103.680985,100.746634,100.598802,102.440505,101.941748,87.499997,98.033716,100.556590,97.903761,100.438543,101.183892,100.000000,95.555560
1,2023-07-26,96.617050,102.395214,100.388687,106.113542,100.512820,98.293236,103.374230,106.383328,97.904194,103.645687,102.314566,97.499995,100.505619,102.782933,99.380661,100.941853,100.769531,100.000000,100.000000
2,2023-07-27,91.069014,98.502995,99.293293,99.563319,95.384614,98.215661,99.693252,106.555618,96.407189,101.868026,101.941748,100.000000,97.078660,98.330247,96.903289,94.563213,101.519336,100.000000,91.111113
3,2023-07-28,96.075779,103.293417,97.950532,104.366819,98.461534,100.659421,101.840489,109.131923,102.994016,101.717372,105.339806,100.000000,97.528097,96.474952,96.426870,95.809046,103.749010,100.000000,93.333333
4,2023-07-31,99.729364,111.377249,99.151945,146.943234,102.051280,100.892155,110.429445,109.213977,108.982042,102.259704,105.339806,87.499997,100.337086,99.257886,99.952357,94.428674,105.268338,100.000000,95.555560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2025-07-21,638.565622,210.479054,678.727224,258.733637,143.141022,490.276860,245.398774,157.219740,520.059919,169.237643,560.504861,6314.999962,142.471920,381.447124,164.840395,134.835333,98.412762,76.805952,784.444459
499,2025-07-22,634.370797,197.604793,681.828249,239.956331,129.102557,486.703024,249.693263,158.009340,489.820390,169.614421,560.504861,6349.999809,145.674165,361.410019,160.600283,135.516976,99.674462,77.248586,791.111119
500,2025-07-23,665.087989,205.389231,721.600658,248.034931,131.025641,505.737628,260.736197,157.515838,506.586842,175.203341,560.504861,6459.999941,151.067419,363.265314,165.221537,138.171756,100.407064,75.832157,828.888915
501,2025-07-24,651.285547,204.790429,707.610037,239.956331,131.602565,508.379136,269.018420,158.905855,479.640745,174.952160,560.504861,6737.500090,185.730353,372.912818,164.363985,140.027027,101.750166,75.655099,800.000000


Plot the indexes

In [154]:
import plotly.graph_objs as go

fig = go.Figure()

for col in index_df.columns:
    if col != 'Date':
        fig.add_trace(go.Scatter(
            x=index_df['Date'],
            y=index_df[col],
            mode='lines',
            name=col,
            hoverinfo='x+y+name',
            line=dict(width=2),
            opacity=0.7
        ))

fig.update_layout(
    title='Indexed Performance of Portfolio Components',
    xaxis_title='Date',
    yaxis_title='Index Value',
    height=800,
    legend_title_text='Symbol',
    hovermode='x unified'
)

# Enable highlight on click using Plotly's legend selection
fig.update_traces(
    selected=dict(marker=dict(opacity=1)),
    unselected=dict(marker=dict(opacity=0.5))
)

fig.show()

Calculate some important metrics for each symbol:

In [155]:
# Calculate important metrics
cov_matrix = history_df.drop(columns=['Date']).cov()
portfolio_df['total_value'] = portfolio_df['quantity'] * portfolio_df['Close']
portfolio_df['weight'] = portfolio_df['total_value'] / portfolio_df['total_value'].sum()
# Assume risk-free rate is 0 for simplicity, or set your own value
# Calculate Sharpe ratio for each symbol
portfolio_df['SR'] = ((portfolio_df['r_bar'] - risk_free_rate))/ np.sqrt(portfolio_df['sigma'])
portfolio_r= portfolio_df['weight'].T @ portfolio_df['r_bar']
# Annualize the portfolio variance by multiplying by 252 (number of trading days)
portfolio_variance = (portfolio_df['weight'].T @ cov_matrix @ portfolio_df['weight'])

# Calculate covariance of each symbol with the portfolio
portfolio_df['cov(i,p)'] = cov_matrix.dot(portfolio_df['weight'])


print ("Portfolio total value: {:.2f}".format(portfolio_df['total_value'].sum()))
print("Portfolio r_bar: {:.2f}%".format(portfolio_r * 100))
print("Portfolio x_bar: {:.2f}%".format((portfolio_r-risk_free_rate) * 100))
print("Portfolio Variance:", "{:.5f}".format(portfolio_variance))
portfolio_sigma=np.sqrt(portfolio_variance) * 100
print ("Portfolio sigma: {:.2f}%".format(portfolio_sigma))
print("Portfolio Sharpe Ratio: {:.2f}".format((portfolio_r-risk_free_rate) / (np.sqrt(portfolio_variance))))

Portfolio total value: 60913.89
Portfolio r_bar: 42.15%
Portfolio x_bar: 41.15%
Portfolio Variance: 0.00009
Portfolio sigma: 0.97%
Portfolio Sharpe Ratio: 42.35


In [133]:
portfolio_df

,quantity,Close,r_bar,sigma,skewness,kurtosis,total_value,weight,SR,"cov(i,p)"
symbol,,,,,,,,,,
RKLB,100,47.430000,0.613384,0.752444,1.466283,9.419981,4743.000031,0.147701,0.695596,0.000973
PL,650,6.650000,0.169479,0.730308,0.915960,16.676961,4322.500062,0.134606,0.186616,0.000892
VST,20,192.199997,0.408710,0.413689,-0.804898,15.229533,3843.999939,0.119705,0.619898,0.000371
ACHR,320,11.210000,0.373067,0.846546,1.057005,5.115588,3587.200012,0.111708,0.394604,0.000969
BKSY,180,19.780001,0.081672,0.853715,4.642329,79.129140,3560.400124,0.110874,0.077570,0.000949
VRT,25,137.470001,0.527303,0.539646,-0.725887,18.215465,3436.750031,0.107023,0.704191,0.000562
UEC,370,8.910000,0.325295,0.799490,1.004074,7.389479,3296.699944,0.102662,0.352623,0.000683
GOOG,15,194.080002,0.255241,0.306875,0.601539,8.952765,2911.200027,0.090657,0.442703,0.000254
RDW,150,16.070000,0.435073,0.841040,1.711556,16.151000,2410.499954,0.075065,0.463505,0.000936
